In [1]:
import numpy as np

a = [[1, 2], [3, 4]]
b = [[5, 6], [7, 8]]

# dot
c=np.dot(a, b)
print(c)

# outer product
d=np.outer(a, b)
print(d)

[[19 22]
 [43 50]]
[[ 5  6  7  8]
 [10 12 14 16]
 [15 18 21 24]
 [20 24 28 32]]


In [3]:
import numpy as np

a=np.array([1,0])
print(a)
b=a.T
print(b)

[1 0]
[1 0]


# 0.Install quantuma software framework

In [ ]:
!pip install pennylane

     |████████████████████████████████| 800 kB 7.0 MB/s 
     |████████████████████████████████| 1.6 MB 48.5 MB/s 
     |████████████████████████████████| 7.8 MB 37.7 MB/s 
     |████████████████████████████████| 108 kB 47.0 MB/s 


# Section 2.2. Example

In [ ]:
#https://pennylane.ai/qml/demos/tutorial_qubit_rotation.html

# 1. import penny lane and numpy
import pennylane as qml
from pennylane import numpy as np


# 2. create device

# "default.qubit": using the qubit model
# wires=1: use a single qubit
dev = qml.device("default.qubit", wires=1)

@qml.qnode(dev)
def circuit(theta, phi):

    # 1. use hadamard gate to transfrom to Sx basis from Sz basis
    qml.Hadamard(wires=[0])

    # 2. rotate analyzer to n direction
    qml.Rot( phi,theta,0,wires=[0])
    
    # 3. get the probabilities of quantum states
    return qml.probs(wires=[0])
pass

phi=np.pi/3.0
theta=np.pi*2.0/3.0
print(circuit(phi,theta))
print(qml.draw(circuit)(phi,theta))

[0.71650635 0.28349365]
 0: ──H──Rot(2.09, 1.05, 0)──┤ Probs 



# Section 4.1. QKD Example

In [ ]:
from pkg_resources import parse_version
#https://pennylane.ai/qml/demos/tutorial_qubit_rotation.html

# 1. import penny lane and numpy
import pennylane as qml
from pennylane import numpy as np
import random

random.seed(6)
np.random.seed(8)


# 2. create device
num_of_bit=6
# "default.qubit": using the qubit model
# wires=1: use a single qubit
dev = qml.device("default.qubit", wires=num_of_bit,shots=1)

@qml.qnode(dev)
def alice_q_circuit(Alice_key_table,Alice_q_basis):


  for i in range(num_of_bit):
    if (Alice_q_basis[i]=="z"):
      if (Alice_key_table[i]==0):
        print(f"{i}-th qbit: basis-{Alice_q_basis[i]} spin- {Alice_key_table[i]} ")
      elif (Alice_key_table[i]==1):
        print(f"{i}-th qbit: basis-{Alice_q_basis[i]} spin- {Alice_key_table[i]} ")
        qml.Rot( 0, np.pi,0,wires=[i])
      pass
    elif(Alice_q_basis[i]=="x"):
      # 1. use hadamard gate to transfrom to Sx basis from Sz basis
      qml.Hadamard(wires=[i])
      if (Alice_key_table[i]==0):
        qml.Rot( 0,3*np.pi/2,0,wires=[i])
        print(f"{i}-th qbit: basis-{Alice_q_basis[i]} spin- {Alice_key_table[i]} ")
      elif (Alice_key_table[i]==1):
        qml.Rot( 0,np.pi/2,0,wires=[i])
        print(f"{i}-th qbit: basis-{Alice_q_basis[i]} spin- {Alice_key_table[i]} ")
      pass
    pass
  pass
  # 1. use hadamard gate to transfrom to Sx basis from Sz basis
  #qml.Hadamard(wires=[0])
  # 2. rotate analyzer to n direction
  #qml.Rot( 0,0,0,wires=[0])
  # 3. get the probabilities of quantum states
  return qml.sample()
pass

@qml.qnode(dev)
def QKD_System(Alice_key_table,Alice_q_basis,Bob_q_basis):
  ##########################
  # Allince side

  # generate alice q bit
  for i in range(num_of_bit):
    if (Alice_q_basis[i]=="z"):
      if (Alice_key_table[i]==0):
        pass #print(f"{i}-th qbit: basis-{Alice_q_basis[i]} spin- {Alice_key_table[i]} ")
      elif (Alice_key_table[i]==1):

        qml.Rot( 0, np.pi,0,wires=[i])
      pass
    elif(Alice_q_basis[i]=="x"):
      # 1. use hadamard gate to transfrom to Sx basis from Sz basis
      #qml.Hadamard(wires=[i])
      if (Alice_key_table[i]==0):
        qml.Rot( 0,3*np.pi/2,0,wires=[i])

      elif (Alice_key_table[i]==1):
        qml.Rot( 0,np.pi/2,0,wires=[i])
      pass
    pass
  pass

  ##########################
  # Allince side


  # bob measure
  for i in range(num_of_bit):

    if(Bob_q_basis[i]=="x"):
      # rotate 90 decree 
      if (Alice_key_table[i]==0):
        #qml.Rot( 0,1*np.pi/2,0,wires=[i])
        qml.Rot( 0,1*np.pi/2,0,wires=[i])
      elif (Alice_key_table[i]==1):
        #qml.Rot( 0,-3*np.pi/2,0,wires=[i])
        qml.Rot( 0,-np.pi/2,0,wires=[i])
      pass
    pass
  pass

  return qml.sample()
pass

# step 1: Allice decide key randomly
Alice_key_table=np.random.choice(2, num_of_bit)
print("# step 1: Alice_key_table: ",Alice_key_table)

# step 2: Allice decide quantum basis randomly
basis = ["z", "x"]
# choose three random sample with replacement to including repetition
Alice_q_basis = random.choices(basis, k=num_of_bit)
print("# step 2: Alice_q_basis: ",Alice_q_basis)

# step 3: Allice send 6 quantum bit without sending quantum basis
allice_send_q_basis=alice_q_circuit(Alice_key_table,Alice_q_basis)
print(f"# Step 3: Allice send qbit  (0=>spin up, 1=> spin down): {allice_send_q_basis}")

# step 4: Bob decide quantum basis randomly
basis = ["z", "x"]
# choose three random sample with replacement to including repetition
Bob_q_basis = random.choices(basis, k=num_of_bit)
print("# step 4: Bob_q_basis: ",Bob_q_basis)

# step 5: Bob measure Allice's quantum basis based on Bob's quantum basis
Bob_measured_qbit=QKD_System(Alice_key_table,Alice_q_basis,Bob_q_basis)
print("# step 5: Bob measured qbit (0=>spin up, 1=> spin down):",Bob_measured_qbit)
      
# step 6: Bob sends Bob's basis to Allice
print("# step 6: Bob sends Bob's basis to Allice:",Bob_q_basis)

# step 7: Allice compares Allice's basis and Bob basis and send back comparsion to Bob
print("# step 7: Allice compares Allice's basis and Bob basis")
print("Alice's basis: \t", Alice_q_basis)
print("Bob's basis: \t", Bob_q_basis)
basis_comparison=[]
for i in range(num_of_bit):
 if Alice_q_basis[i]==Bob_q_basis[i]:
   basis_comparison.append(True)
 else:
   basis_comparison.append(False)
 pass
pass
print("Comparison: \t",basis_comparison)

#Step 9: Bob picks up avaiable qbits, then separates them into checking bits and key bits, and sends the checking bits to AliceBob_checking_bit
print("#Step 9: Bob picks up avaiable qbits, then separates them into checking bits and key bits, and sends the checking bits to AliceBob_checking_bit")
Bob_available_bit={}
for i in range(num_of_bit):
 if (basis_comparison[i]==True):
  Bob_available_bit[i]=Bob_measured_qbit[i]
 pass
pass
print("Bob's Available bit: \t",Bob_available_bit)

# pick the 1st, 4th bit as checking bit
Bob_checking_bit={}
Bob_checking_bit[1]=Bob_available_bit[1]
Bob_checking_bit[4]=Bob_available_bit[4]
print("Bob Checking bit: \t",Bob_checking_bit)

# pick the 2nd, 5th bit as key bit
Bob_key_bit={}
Bob_key_bit[2]=Bob_available_bit[2]
Bob_key_bit[5]=Bob_available_bit[5]
print("Bob Key bit: \t\t",Bob_key_bit)

# send back checking bit to Alice
print("Bob sends back Checking bit to Alice: \t",Bob_checking_bit)

# Step 10: Allice check the checking bit from Bob
print("# Step 10: Allice check the checking bit from Bob")

is_checking_bit_fine=True
for key, value in Bob_checking_bit.items():
 if Alice_key_table[key]!=Bob_checking_bit[key]:
   is_checking_bit_fine=False
 pass
 print(f"Alice_key_table[{key}]={Alice_key_table[key]} vs Bob_checking_bit[{key}]={Bob_checking_bit[key]} == {Alice_key_table[key]==Bob_checking_bit[key]} ")
pass

Alice_key_bit={}
if is_checking_bit_fine:
  print("Allice's checking result is pass.")

  # get alice key bit
  for i in range(num_of_bit):
    if (basis_comparison[i]==True):
      Alice_key_bit[i]=Alice_key_table[i]
    pass
  pass
  for key, value in Bob_checking_bit.items():
   del Alice_key_bit[key]
  pass
  print("Allice's key bit is:", Alice_key_bit)
else:
  print("Allice's checking result is not pass.")
pass


# step 1: Alice_key_table:  [1 0 1 1 1 0]
# step 2: Alice_q_basis:  ['x', 'x', 'z', 'z', 'z', 'x']
0-th qbit: basis-x spin- 1 
1-th qbit: basis-x spin- 0 
2-th qbit: basis-z spin- 1 
3-th qbit: basis-z spin- 1 
4-th qbit: basis-z spin- 1 
5-th qbit: basis-x spin- 0 
# Step 3: Allice send qbit  (0=>spin up, 1=> spin down): [1 0 1 1 1 0]
# step 4: Bob_q_basis:  ['z', 'x', 'z', 'x', 'z', 'x']
# step 5: Bob measured qbit (0=>spin up, 1=> spin down): [0 0 1 0 1 0]
# step 6: Bob sends Bob's basis to Allice: ['z', 'x', 'z', 'x', 'z', 'x']
# step 7: Allice compares Allice's basis and Bob basis
Alice's basis: 	 ['x', 'x', 'z', 'z', 'z', 'x']
Bob's basis: 	 ['z', 'x', 'z', 'x', 'z', 'x']
Comparison: 	 [False, True, True, False, True, True]
#Step 9: Bob picks up avaiable qbits, then separates them into checking bits and key bits, and sends the checking bits to AliceBob_checking_bit
Bob's Available bit: 	 {1: tensor(0, requires_grad=True), 2: tensor(1, requires_grad=True), 4: tensor(1, requires_gr